<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/transformers-for-natural-language-processing/9-semantic-role-labeling-with-bert-based-transformers/semantic_role_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Semantic Role Labeling(SRL)


SRL labels the semantic role as the role a word or group of words plays in a sentence and the relationship established with the predicate.

A semantic role is the role a noun or noun phrase plays in relation to the main verb in a sentence.

The predicate describes something about the subject or agent. The predicate could be anything
that provides information on the features or actions of a subject.

The noun or noun phrases that revolve around the predicate are arguments or argument terms.




We will run our SRL experiments using the BERT SRL. We will begin with basic samples with various sentence structures. We will then challenge the BERT-based model with some more difficult samples to explore the system's capacity and limits.

The notebook is an implementation of the Allen Institute for AI BERT-based model. [Reference usage of the Notebook](https://demo.allennlp.org/semantic-role-labeling/MjE4NjI1Ng==)

The BERT-based model is an implementation of [Peng Shi and Jimmy Lin, (2019), ‘Simple BERT Models for Relation Extraction and Semantic Role Labeling’]( https://arxiv.org/abs/1904.05255)

## Setup

In [ ]:
!pip install -q allennlp==2.1.0 allennlp-models==2.1.0

In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import json

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

We also add two functions to display the JSON object SRL BERT returns.

In [4]:
def head(prediction):
  # Iterating through the json to display excerpt of the prediciton
  for i in prediction['verbs']:
    print('Verb:',i['verb'],i['description'])

def full(prediction):
  #print the full prediction
  print(json.dumps(prediction, indent = 1, sort_keys=True))

##Basic samples

Basic samples seem intuitively simple but can be tricky to analyze. Compound
sentences, adjectives, adverbs, and modals are not easy to identify, even for nonexpert humans.

Let's begin with an easy sample for the transformer.

###Sample 1

The first sample is long but relatively easy for the transformer:

In [5]:
prediction = predictor.predict(sentence="Did Bob really think he could prepare a meal for 50 people in only a few hours?")
head(prediction)

Verb: Did [V: Did] Bob really think he could prepare a meal for 50 people in only a few hours ?
Verb: think Did [ARG0: Bob] [ARGM-ADV: really] [V: think] [ARG1: he could prepare a meal for 50 people in only a few hours] ?
Verb: could Did Bob really think he [V: could] [ARG1: prepare a meal for 50 people in only a few hours] ?
Verb: prepare Did Bob really think [ARG0: he] [ARGM-MOD: could] [V: prepare] [ARG1: a meal for 50 people] [ARGM-TMP: in only a few hours] ?


In [6]:
full(prediction)

{
 "verbs": [
  {
   "description": "[V: Did] Bob really think he could prepare a meal for 50 people in only a few hours ?",
   "tags": [
    "B-V",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O"
   ],
   "verb": "Did"
  },
  {
   "description": "Did [ARG0: Bob] [ARGM-ADV: really] [V: think] [ARG1: he could prepare a meal for 50 people in only a few hours] ?",
   "tags": [
    "O",
    "B-ARG0",
    "B-ARGM-ADV",
    "B-V",
    "B-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
    "O"
   ],
   "verb": "think"
  },
  {
   "description": "Did Bob really think he [V: could] [ARG1: prepare a meal for 50 people in only a few hours] ?",
   "tags": [
    "O",
    "O",
    "O",
    "O",
    "O",
    "B-V",
    "B-ARG1",
    "I-ARG1",
    "I-ARG1",
    "I-ARG1",
   

###Sample 2

In [7]:
prediction = predictor.predict(sentence="Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower.")
head(prediction)

Verb: went [ARG0: Mrs. and Mr. Tomaso] [V: went] [ARG4: to Europe] [ARGM-PRP: for vacation] and visited Paris and first went to visit the Eiffel Tower .
Verb: visited [ARG0: Mrs. and Mr. Tomaso] went to Europe for vacation and [V: visited] [ARG1: Paris] and first went to visit the Eiffel Tower .
Verb: went [ARG0: Mrs. and Mr. Tomaso] went to Europe for vacation and visited Paris and [ARGM-TMP: first] [V: went] [ARG1: to visit the Eiffel Tower] .
Verb: visit [ARG0: Mrs. and Mr. Tomaso] went to Europe for vacation and visited Paris and first went to [V: visit] [ARG1: the Eiffel Tower] .


###Sample 3

Now we will will make things more difficult for our transformer model. The following sample contains the verb "drink" four times:

```John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice.```

In [8]:
prediction = predictor.predict(sentence="John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice.")
head(prediction)

Verb: wanted [ARG0: John] [V: wanted] [ARG1: to drink tea] , Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice .
Verb: drink [ARG0: John] wanted to [V: drink] [ARG1: tea] , Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice .
Verb: likes John wanted to drink tea , [ARG0: Mary] [V: likes] [ARG1: to drink coffee] but Karim drank some cool water and Faiza would like to drink tomato juice .
Verb: drink John wanted to drink tea , [ARG0: Mary] likes to [V: drink] [ARG1: coffee] but Karim drank some cool water and Faiza would like to drink tomato juice .
Verb: drank John wanted to drink tea , Mary likes to drink coffee but [ARG0: Karim] [V: drank] [ARG1: some cool water] and Faiza would like to drink tomato juice .
Verb: would John wanted to drink tea , Mary likes to drink coffee but Karim drank some cool water and [ARG0: Faiza] [V: would] like [ARG1: to drink tomato juice] .
Verb: like John

##Difficult samples

Let's start with a complex sample that the BERT-based transformer can analyze.

###Sample 4

It takes us into more tricky SRL territory. The sample separates "Alice" from
the verb "liked," creating a long-term dependency that has to jump over "whose
husband went jogging every Sunday."

The sentence is:

```Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime.```

A human can isolate "Alice" and find the predicate:

```
[Alice, whose husband went jogging every Sunday], liked to go to a dancing
class in the meantime.```

Can the BERT model find the predicate like us?

In [9]:
prediction = predictor.predict(sentence="Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime.")
head(prediction)

Verb: went Alice , [ARG0: whose husband] [V: went] [ARG1: jogging] [ARGM-TMP: every Sunday] , liked to go to a dancing class in the meantime .
Verb: jogging Alice , [ARG0: whose husband] went [V: jogging] [ARGM-TMP: every Sunday] , liked to go to a dancing class in the meantime .
Verb: liked [ARG0: Alice , whose husband went jogging every Sunday] , [V: liked] [ARG1: to go to a dancing class in the meantime] .
Verb: go [ARG0: Alice , whose husband went jogging every Sunday] , liked to [V: go] [ARG4: to a dancing class] [ARGM-TMP: in the meantime] .
Verb: dancing Alice , whose husband went jogging every Sunday , liked to go to a [V: dancing] class in the meantime .


###Sample 5

Sample 5 does not repeat a verb several times. However, Sample 5 contains a word
that can have multiple functions and meanings. It goes beyond polysemy since
the word "round" can have both different meanings and grammatical functions.
The word "round" can be a noun, an adjective, an adverb, a transitive verb, or an intransitive verb.

In [11]:
prediction = predictor.predict(sentence="The bright sun, the blue sky, the warm sand, the palm trees, everything round off.")
head(prediction)

In [12]:
full(prediction)

{
 "verbs": [],
 "words": [
  "The",
  "bright",
  "sun",
  ",",
  "the",
  "blue",
  "sky",
  ",",
  "the",
  "warm",
  "sand",
  ",",
  "the",
  "palm",
  "trees",
  ",",
  "everything",
  "round",
  "off",
  "."
 ]
}


The output shows no verbs. The transformer did not identify the predicate. In fact, it found no verbs at all.

Since we like our BERT-based transformer, we will be kind to it. 

Let's change the sentence from the past tense to the present tense.



In [13]:
prediction = predictor.predict(sentence="The bright sun, the blue sky, the warm sand, the palm trees, everything rounds off.")
head(prediction)

Verb: rounds [ARG1: The bright sun , the blue sky , the warm sand , the palm trees , everything] [V: rounds] [ARGM-PRD: off] .


###Sample 6

Sample 6 takes a word we often think is just a noun. However, more words than
we suspect can be both nouns and verbs. "To ice" is a verb used in hockey to shoot a "puck" all the way across the rink and beyond the goal line of an opponent. The "puck" is the disk used in hockey.

A hockey coach can start the day by telling a team to train icing pucks. We then can obtain the imperative sentence when the coach yells:

```Now, ice pucks guys!```

Note that "guys" can mean "persons" regardless of their sex.

In [14]:
prediction = predictor.predict(sentence="Now, ice pucks guys!")
head(prediction)

In [15]:
full(prediction)

{
 "verbs": [],
 "words": [
  "Now",
  ",",
  "ice",
  "pucks",
  "guys",
  "!"
 ]
}


Game over! We can see that transformers have made tremendous progress, but there
is still a lot of room for developers to improve the models. Humans are still in the game!